In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Data Understanding

In [101]:
df = pd.read_csv('housePrice.csv', encoding='ISO-8859-1')


In [102]:
df.head(5)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33


In [103]:
df = df.drop("Price", axis = 1)

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        3479 non-null   object 
 1   Room        3479 non-null   int64  
 2   Parking     3479 non-null   bool   
 3   Warehouse   3479 non-null   bool   
 4   Elevator    3479 non-null   bool   
 5   Address     3456 non-null   object 
 6   Price(USD)  3479 non-null   float64
dtypes: bool(3), float64(1), int64(1), object(2)
memory usage: 119.0+ KB


In [105]:
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')


In [106]:
df.isnull().sum()

Area           6
Room           0
Parking        0
Warehouse      0
Elevator       0
Address       23
Price(USD)     0
dtype: int64

The number of nulls is much less than 5% of the data, so it is best to delete those rows, as it has little negative impact on the model and the data becomes cleaner.

In [107]:
df = df.dropna()

In [108]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Area,3450.0,106.917391,69.550976,30.0,69.0000,90.00,120.0,929.0
Room,3450.0,2.081159,0.760216,0.0,2.0000,2.00,2.0,5.0
Price(USD),3450.0,179185.420345,270863.932158,120.0,47308.3325,96666.67,200000.0,3080000.0


In [109]:
df['Address'].value_counts()

Address
Punak                     161
Pardis                    146
West Ferdows Boulevard    145
Gheitarieh                140
Shahran                   130
                         ... 
Chardangeh                  1
Mehrabad                    1
Pakdasht KhatunAbad         1
Kazemabad                   1
Yakhchiabad                 1
Name: count, Length: 192, dtype: int64

In [113]:
df['Avg_Price_By_address'] = df.groupby('Address')['Price(USD)'].transform('mean')
df.sample(8)

,Area,Room,Parking,Warehouse,Elevator,Address,Price(USD),Avg_Price_By_address
1346,100.0,2,True,True,True,Niavaran,206666.67,663106.862794
687,102.0,2,True,True,True,Persian Gulf Martyrs Lake,150000.00,102752.651250
2144,75.0,2,False,False,False,Baghestan,20000.00,23562.501250
960,105.0,2,True,True,True,Koohsar,133333.33,110416.665000
2810,200.0,3,True,True,True,Shahrake Gharb,466666.67,398996.045085
2587,80.0,2,True,True,False,Islamshahr,36666.67,41914.102308
2595,113.0,2,True,True,False,Islamshahr,50666.67,41914.102308
1305,105.0,2,True,True,True,Pardis,17966.67,25927.397329
